In [1]:
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

import tensorflow as tf
import numpy as np

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

#X_train = X_train - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]
#X_test = X_test - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]

X_train_full, y_train_full = X_train, y_train

X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]



c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\ferbi\appd

In [2]:
import tensorflow as tf
import numpy as np
from datetime import datetime
import math

def execute_code(rand_seed = 1, lambda_exp = -6.0):
    LOC = """

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

print("Now:", now)

n_inputs = X_train.shape[1]
n_hidden1 = 1024
n_hidden2 = 1024
n_hidden3 = 1024
n_hidden4 = 1024
n_hidden5 = 1024
n_outputs = 10

def reset_graph(seed=43):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph(@rand_seed@)

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

epsVal = pow(10.0, -8.0)

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2/np.sqrt(n_inputs + n_neurons)
        init = tf.random_normal((n_inputs, n_neurons), mean=0.0, stddev=stddev)
        
        W = tf.Variable(init, name="kernel")
        #W = tf.cast(tf.abs(W) >= epsVal, W.dtype) * W
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.add(tf.matmul(X, W), b)
                
        if activation is not None:
            return activation(Z)
        else:
            return Z
        
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    hidden3 = neuron_layer(hidden2, n_hidden3, name="hidden3", activation=tf.nn.relu)
    hidden4 = neuron_layer(hidden3, n_hidden4, name="hidden4", activation=tf.nn.relu)
    hidden5 = neuron_layer(hidden4, n_hidden5, name="hidden5", activation=tf.nn.relu)
    logits = neuron_layer(hidden5, n_outputs, name="outputs")

hidden1_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden1/kernel:0")
hidden2_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden2/kernel:0")
hidden3_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden3/kernel:0")
hidden4_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden4/kernel:0")
hidden5_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden5/kernel:0")
logits_W = tf.get_default_graph().get_tensor_by_name("dnn/outputs/kernel:0")
    
def calcRegValue(W, type=1, eps=0.00001):
    if type == 1:
        wMax = 1.0
        gt_eps = tf.minimum(1.0, tf.add(1.0, tf.constant(eps/(wMax-eps))*tf.subtract(tf.abs(W), wMax)))
        #gt_eps = 1.0  
        #lt_eps = tf.constant((1-eps)/eps)*tf.abs(W)
        lt_eps = 0.0
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*gt_eps + tf.cast(tf.abs(W) < eps, W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 2:
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*W
        return tf.reduce_sum(reg_value)
    elif type == 3:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.pow(W, 2))))
    elif type == 4:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.abs(W))))
    elif type == 5:
        return tf.reduce_sum(tf.abs(W))
    elif type == 6:
        return tf.reduce_sum(tf.pow(W, 2))
    elif type == 7:
        gt_eps = 1.0  
        epsMat = tf.constant(eps, shape=W.get_shape())
        lt_eps = 1.0 - tf.pow(epsMat, tf.abs(W))
        reg_value = tf.cast(tf.abs(W) >= np.sqrt(eps), W.dtype)*gt_eps + tf.cast(tf.abs(W) < np.sqrt(eps), W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 8:
        eMat = tf.constant(math.e, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(eMat, tf.subtract(0.0, 50*tf.pow(W, 2)))))
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    regType = 8
    total_regvalue = tf.add_n([calcRegValue(hidden1_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden2_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden3_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden4_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden5_W, type=regType, eps=epsVal), \
                               calcRegValue(logits_W, type=regType, eps=epsVal)])
    
    total_nzw = tf.cast(tf.add_n([tf.count_nonzero(hidden1_W), \
                                  tf.count_nonzero(hidden2_W), \
                                  tf.count_nonzero(hidden3_W), \
                                  tf.count_nonzero(hidden4_W), \
                                  tf.count_nonzero(hidden5_W), \
                                  tf.count_nonzero(logits_W)]), tf.float32)
    total_nzw_eps = tf.cast(tf.add_n([tf.reduce_sum(tf.cast(abs(hidden1_W) < epsVal, hidden1_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden2_W) < epsVal, hidden2_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden3_W) < epsVal, hidden3_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden4_W) < epsVal, hidden4_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden5_W) < epsVal, hidden5_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(logits_W) < epsVal, logits_W.dtype))]), tf.int64)
    
    lambdaParam = tf.pow(10.0, @lambda_exp@, name="lambdaParam")
    loss_total = tf.add(tf.reduce_mean(xentropy, name="loss_total"), tf.scalar_mul(lambdaParam, total_regvalue))
    #loss_total = tf.reduce_mean(xentropy, name="loss_total")
    
learning_rate = tf.placeholder(tf.float32, shape=(), name="learning_rate")
with tf.name_scope("train"):
    #optimizer = tf.train.ProximalGradientDescentOptimizer(learning_rate)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    #optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
    #optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    training_op = optimizer.minimize(loss_total)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

reg_summary = tf.summary.scalar('Count_Reg', total_regvalue)
loss_summary = tf.summary.scalar('Loss_Total', loss_total)
err_summary = tf.summary.scalar('Class_Err_Perc', (1-accuracy)*100)
#file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_epochs = 250
batch_size = 20

def shuffle_batch(X, y, batch_size):
    np.random.seed(43)
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    batch_num = -1
    for batch_idx in np.array_split(rnd_idx, n_batches):
        batch_num = batch_num + 1
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield batch_num, X_batch, y_batch

def get_lr_step(epochCounter, start_lr, stepsize, end_lr, lr_iter_period):
    lr = start_lr - math.floor(epochCounter/lr_iter_period) * stepsize
    return max(lr, end_lr)
        
def get_lr_triangle(epochCounter, lr_min, lr_max, lr_iter_period):
    cycle = math.floor(1 + epochCounter/(2*lr_iter_period))
    x = abs(epochCounter/lr_iter_period - 2*cycle + 1)
    lr = lr_min + (lr_max - lr_min) * max(0, (1-x)/math.pow(2, cycle-1))
    return lr

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

start_time = datetime.now()

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        lr = get_lr_triangle(epoch, lr_min=0.01, lr_max=0.25, lr_iter_period=20)
        #lr=0.01
        rnd_idx = np.random.permutation(len(X_train))
        for rnd_indices in np.array_split(rnd_idx, len(X_train) // batch_size):
            X_batch, y_batch = X_train[rnd_indices], y_train[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, learning_rate: lr})
        
        loss_val, reg_loss_val, lambda1, acc_val = sess.run([loss_total, total_regvalue, lambdaParam, accuracy], \
                                                   feed_dict={X: X_valid, y: y_valid, learning_rate: lr})
        base_loss_val = loss_val - lambda1*reg_loss_val
        
        if best_loss > loss_val:
            save_path = saver.save(sess, "./models/model_" + now + ".ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation losses: {:.4f}, {:.4f}, {:.4f}\tBest Loss: {:.4f} ({})\tAccuracy: {:.2f}%".format(
            epoch, base_loss_val, lambda1*reg_loss_val, loss_val, best_loss, checks_without_progress, acc_val * 100))

end_time = datetime.now()
running_time = end_time - start_time
print("Total running time:\t", running_time.seconds)

with tf.Session() as sess:
    saver.restore(sess, "./models/model_" + now + ".ckpt") # or better, use save_path
    #saver.restore(sess, "./models/model_20190129210937.ckpt") # or better, use save_path
    n_batches = len(X_test) // 2000
    batch_remaining = n_batches
    correct_count = 0.0
    for batch_idx in np.array_split(range(len(X_test)), n_batches):
        correct_count = correct_count + len(batch_idx)*accuracy.eval(feed_dict={X: X_test[batch_idx], y: y_test[batch_idx]})
        print("Batches remaining:    ", batch_remaining)
        batch_remaining = batch_remaining - 1
print("Test error rate:    {:.10f}%".format(100*(1-correct_count/len(X_test)))) 
"""
    LOC = LOC.replace("@rand_seed@", str(rand_seed))
    LOC = LOC.replace("@lambda_exp@", str(lambda_exp))
    exec(LOC)

In [3]:
for j in [-5.8, -6.0, -6.2, -6.4, -6.6, -6.8, -7.0]:
    for i in [1, 2, 3]:
        print("RANDOM SEED:  ", i, "\tLAMBDA_EXP:  ", j)
        execute_code(rand_seed=i, lambda_exp=j)

RANDOM SEED:   1 	LAMBDA_EXP:   -5.8
Now: 20240303232946
0	Validation losses: 0.4059, 0.6881, 1.0939	Best Loss: 1.0939 (0)	Accuracy: 84.80%
1	Validation losses: 0.3567, 0.6807, 1.0374	Best Loss: 1.0374 (0)	Accuracy: 87.28%
2	Validation losses: 0.3672, 0.6693, 1.0366	Best Loss: 1.0366 (0)	Accuracy: 86.50%
3	Validation losses: 0.3461, 0.6544, 1.0005	Best Loss: 1.0005 (0)	Accuracy: 87.24%
4	Validation losses: 0.3311, 0.6359, 0.9670	Best Loss: 0.9670 (0)	Accuracy: 87.62%
5	Validation losses: 0.3905, 0.6146, 1.0051	Best Loss: 0.9670 (1)	Accuracy: 86.34%
6	Validation losses: 0.2862, 0.5905, 0.8767	Best Loss: 0.8767 (0)	Accuracy: 89.22%
7	Validation losses: 0.3209, 0.5643, 0.8853	Best Loss: 0.8767 (1)	Accuracy: 88.18%
8	Validation losses: 0.3054, 0.5363, 0.8417	Best Loss: 0.8417 (0)	Accuracy: 88.70%
9	Validation losses: 0.3262, 0.5073, 0.8335	Best Loss: 0.8335 (0)	Accuracy: 88.72%
10	Validation losses: 0.3020, 0.4774, 0.7794	Best Loss: 0.7794 (0)	Accuracy: 89.20%
11	Validation losses: 0.2954,

40	Validation losses: 0.4463, 0.1272, 0.5735	Best Loss: 0.4563 (11)	Accuracy: 90.86%
41	Validation losses: 0.4579, 0.1264, 0.5843	Best Loss: 0.4563 (12)	Accuracy: 90.54%
42	Validation losses: 0.4533, 0.1254, 0.5787	Best Loss: 0.4563 (13)	Accuracy: 90.66%
43	Validation losses: 0.4676, 0.1242, 0.5919	Best Loss: 0.4563 (14)	Accuracy: 90.82%
44	Validation losses: 0.4378, 0.1230, 0.5609	Best Loss: 0.4563 (15)	Accuracy: 90.26%
45	Validation losses: 0.4676, 0.1220, 0.5896	Best Loss: 0.4563 (16)	Accuracy: 89.92%
46	Validation losses: 0.4318, 0.1210, 0.5528	Best Loss: 0.4563 (17)	Accuracy: 89.90%
47	Validation losses: 0.3999, 0.1201, 0.5200	Best Loss: 0.4563 (18)	Accuracy: 90.36%
48	Validation losses: 0.4094, 0.1195, 0.5289	Best Loss: 0.4563 (19)	Accuracy: 90.36%
49	Validation losses: 0.3966, 0.1193, 0.5159	Best Loss: 0.4563 (20)	Accuracy: 89.94%
Early stopping!
Total running time:	 4086
INFO:tensorflow:Restoring parameters from ./models/model_20240304004151.ckpt
Batches remaining:     5
Batche

30	Validation losses: 0.3177, 0.1833, 0.5010	Best Loss: 0.5010 (0)	Accuracy: 90.36%
31	Validation losses: 0.3535, 0.1788, 0.5322	Best Loss: 0.5010 (1)	Accuracy: 90.46%
32	Validation losses: 0.3509, 0.1747, 0.5255	Best Loss: 0.5010 (2)	Accuracy: 90.78%
33	Validation losses: 0.3697, 0.1710, 0.5407	Best Loss: 0.5010 (3)	Accuracy: 89.90%
34	Validation losses: 0.3594, 0.1677, 0.5271	Best Loss: 0.5010 (4)	Accuracy: 90.70%
35	Validation losses: 0.3716, 0.1648, 0.5364	Best Loss: 0.5010 (5)	Accuracy: 90.72%
36	Validation losses: 0.4083, 0.1623, 0.5706	Best Loss: 0.5010 (6)	Accuracy: 90.30%
37	Validation losses: 0.4196, 0.1603, 0.5800	Best Loss: 0.5010 (7)	Accuracy: 90.82%
38	Validation losses: 0.4526, 0.1588, 0.6114	Best Loss: 0.5010 (8)	Accuracy: 91.08%
39	Validation losses: 0.4717, 0.1578, 0.6295	Best Loss: 0.5010 (9)	Accuracy: 90.70%
40	Validation losses: 0.4896, 0.1573, 0.6470	Best Loss: 0.5010 (10)	Accuracy: 91.00%
41	Validation losses: 0.5026, 0.1566, 0.6592	Best Loss: 0.5010 (11)	Accurac

19	Validation losses: 0.3187, 0.2514, 0.5701	Best Loss: 0.5701 (0)	Accuracy: 89.34%
20	Validation losses: 0.3589, 0.2415, 0.6003	Best Loss: 0.5701 (1)	Accuracy: 87.40%
21	Validation losses: 0.3570, 0.2320, 0.5891	Best Loss: 0.5701 (2)	Accuracy: 87.66%
22	Validation losses: 0.3614, 0.2230, 0.5844	Best Loss: 0.5701 (3)	Accuracy: 88.52%
23	Validation losses: 0.3055, 0.2147, 0.5202	Best Loss: 0.5202 (0)	Accuracy: 89.44%
24	Validation losses: 0.3175, 0.2064, 0.5239	Best Loss: 0.5202 (1)	Accuracy: 89.44%
25	Validation losses: 0.3262, 0.1990, 0.5252	Best Loss: 0.5202 (2)	Accuracy: 89.88%
26	Validation losses: 0.3335, 0.1926, 0.5261	Best Loss: 0.5202 (3)	Accuracy: 89.02%
27	Validation losses: 0.3288, 0.1866, 0.5155	Best Loss: 0.5155 (0)	Accuracy: 89.66%
28	Validation losses: 0.3247, 0.1813, 0.5060	Best Loss: 0.5060 (0)	Accuracy: 90.32%
29	Validation losses: 0.3149, 0.1762, 0.4911	Best Loss: 0.4911 (0)	Accuracy: 90.18%
30	Validation losses: 0.3644, 0.1717, 0.5361	Best Loss: 0.4911 (1)	Accuracy:

15	Validation losses: 0.3051, 0.2301, 0.5352	Best Loss: 0.5352 (0)	Accuracy: 89.46%
16	Validation losses: 0.3542, 0.2259, 0.5801	Best Loss: 0.5352 (1)	Accuracy: 88.04%
17	Validation losses: 0.3132, 0.2214, 0.5346	Best Loss: 0.5346 (0)	Accuracy: 88.76%
18	Validation losses: 0.3298, 0.2174, 0.5472	Best Loss: 0.5346 (1)	Accuracy: 89.16%
19	Validation losses: 0.3336, 0.2135, 0.5471	Best Loss: 0.5346 (2)	Accuracy: 89.02%
20	Validation losses: 0.3353, 0.2097, 0.5450	Best Loss: 0.5346 (3)	Accuracy: 88.38%
21	Validation losses: 0.3508, 0.2050, 0.5558	Best Loss: 0.5346 (4)	Accuracy: 88.96%
22	Validation losses: 0.3419, 0.2009, 0.5428	Best Loss: 0.5346 (5)	Accuracy: 89.42%
23	Validation losses: 0.3638, 0.1968, 0.5606	Best Loss: 0.5346 (6)	Accuracy: 88.88%
24	Validation losses: 0.3210, 0.1928, 0.5138	Best Loss: 0.5138 (0)	Accuracy: 89.94%
25	Validation losses: 0.3159, 0.1886, 0.5045	Best Loss: 0.5045 (0)	Accuracy: 90.18%
26	Validation losses: 0.3522, 0.1846, 0.5368	Best Loss: 0.5045 (1)	Accuracy:

11	Validation losses: 0.2913, 0.1682, 0.4596	Best Loss: 0.4596 (0)	Accuracy: 89.44%
12	Validation losses: 0.3868, 0.1675, 0.5543	Best Loss: 0.4596 (1)	Accuracy: 86.92%
13	Validation losses: 0.3178, 0.1668, 0.4845	Best Loss: 0.4596 (2)	Accuracy: 88.58%
14	Validation losses: 0.3113, 0.1663, 0.4776	Best Loss: 0.4596 (3)	Accuracy: 89.16%
15	Validation losses: 0.3414, 0.1656, 0.5070	Best Loss: 0.4596 (4)	Accuracy: 88.38%
16	Validation losses: 0.3168, 0.1648, 0.4816	Best Loss: 0.4596 (5)	Accuracy: 89.02%
17	Validation losses: 0.3338, 0.1642, 0.4981	Best Loss: 0.4596 (6)	Accuracy: 88.26%
18	Validation losses: 0.3245, 0.1693, 0.4938	Best Loss: 0.4596 (7)	Accuracy: 89.22%
19	Validation losses: 0.3446, 0.1691, 0.5137	Best Loss: 0.4596 (8)	Accuracy: 88.68%
20	Validation losses: 0.3389, 0.1689, 0.5078	Best Loss: 0.4596 (9)	Accuracy: 88.90%
21	Validation losses: 0.3817, 0.1675, 0.5492	Best Loss: 0.4596 (10)	Accuracy: 88.04%
22	Validation losses: 0.3317, 0.1661, 0.4978	Best Loss: 0.4596 (11)	Accurac

24	Validation losses: 0.3566, 0.1579, 0.5144	Best Loss: 0.4581 (16)	Accuracy: 88.82%
25	Validation losses: 0.3618, 0.1562, 0.5180	Best Loss: 0.4581 (17)	Accuracy: 88.90%
26	Validation losses: 0.3558, 0.1547, 0.5104	Best Loss: 0.4581 (18)	Accuracy: 89.48%
27	Validation losses: 0.3509, 0.1530, 0.5039	Best Loss: 0.4581 (19)	Accuracy: 89.74%
28	Validation losses: 0.3616, 0.1514, 0.5131	Best Loss: 0.4581 (20)	Accuracy: 90.58%
Early stopping!
Total running time:	 2405
INFO:tensorflow:Restoring parameters from ./models/model_20240304111843.ckpt
Batches remaining:     5
Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remaining:     1
Test error rate:    11.4000010490%
RANDOM SEED:   1 	LAMBDA_EXP:   -6.6
Now: 20240304115850
0	Validation losses: 0.4033, 0.1096, 0.5129	Best Loss: 0.5129 (0)	Accuracy: 84.90%
1	Validation losses: 0.3498, 0.1096, 0.4595	Best Loss: 0.4595 (0)	Accuracy: 87.60%
2	Validation losses: 0.3637, 0.1097, 0.4733	Best Loss: 0.4595 (1)	Accurac

25	Validation losses: 0.3318, 0.1163, 0.4481	Best Loss: 0.4097 (17)	Accuracy: 89.78%
26	Validation losses: 0.3292, 0.1159, 0.4452	Best Loss: 0.4097 (18)	Accuracy: 89.88%
27	Validation losses: 0.3336, 0.1155, 0.4491	Best Loss: 0.4097 (19)	Accuracy: 89.14%
28	Validation losses: 0.3579, 0.1150, 0.4729	Best Loss: 0.4097 (20)	Accuracy: 90.60%
Early stopping!
Total running time:	 2405
INFO:tensorflow:Restoring parameters from ./models/model_20240304131621.ckpt
Batches remaining:     5
Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remaining:     1
Test error rate:    12.0699989796%
RANDOM SEED:   1 	LAMBDA_EXP:   -6.8
Now: 20240304135627
0	Validation losses: 0.4021, 0.0692, 0.4713	Best Loss: 0.4713 (0)	Accuracy: 85.02%
1	Validation losses: 0.3530, 0.0693, 0.4223	Best Loss: 0.4223 (0)	Accuracy: 87.50%
2	Validation losses: 0.3544, 0.0694, 0.4237	Best Loss: 0.4223 (1)	Accuracy: 86.94%
3	Validation losses: 0.3528, 0.0695, 0.4223	Best Loss: 0.4223 (2)	Accuracy:

28	Validation losses: 0.4087, 0.0815, 0.4902	Best Loss: 0.3703 (20)	Accuracy: 89.80%
Early stopping!
Total running time:	 2409
INFO:tensorflow:Restoring parameters from ./models/model_20240304151121.ckpt
Batches remaining:     5
Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remaining:     1
Test error rate:    11.5499997139%
RANDOM SEED:   1 	LAMBDA_EXP:   -7.0
Now: 20240304155131
0	Validation losses: 0.3987, 0.0437, 0.4424	Best Loss: 0.4424 (0)	Accuracy: 85.24%
1	Validation losses: 0.3545, 0.0437, 0.3982	Best Loss: 0.3982 (0)	Accuracy: 87.18%
2	Validation losses: 0.3564, 0.0438, 0.4003	Best Loss: 0.3982 (1)	Accuracy: 86.80%
3	Validation losses: 0.3420, 0.0440, 0.3860	Best Loss: 0.3860 (0)	Accuracy: 87.28%
4	Validation losses: 0.3257, 0.0441, 0.3699	Best Loss: 0.3699 (0)	Accuracy: 87.84%
5	Validation losses: 0.4146, 0.0444, 0.4590	Best Loss: 0.3699 (1)	Accuracy: 85.18%
6	Validation losses: 0.3002, 0.0446, 0.3448	Best Loss: 0.3448 (0)	Accuracy: 88.92

21	Validation losses: 0.3446, 0.0533, 0.3978	Best Loss: 0.3441 (13)	Accuracy: 88.50%
22	Validation losses: 0.4437, 0.0538, 0.4975	Best Loss: 0.3441 (14)	Accuracy: 88.38%
23	Validation losses: 0.3463, 0.0542, 0.4005	Best Loss: 0.3441 (15)	Accuracy: 89.66%
24	Validation losses: 0.3602, 0.0545, 0.4147	Best Loss: 0.3441 (16)	Accuracy: 89.34%
25	Validation losses: 0.3514, 0.0547, 0.4061	Best Loss: 0.3441 (17)	Accuracy: 89.72%
26	Validation losses: 0.3782, 0.0549, 0.4331	Best Loss: 0.3441 (18)	Accuracy: 89.64%
27	Validation losses: 0.3869, 0.0550, 0.4420	Best Loss: 0.3441 (19)	Accuracy: 89.86%
28	Validation losses: 0.4337, 0.0551, 0.4888	Best Loss: 0.3441 (20)	Accuracy: 90.42%
Early stopping!
Total running time:	 2407
INFO:tensorflow:Restoring parameters from ./models/model_20240304171953.ckpt
Batches remaining:     5
Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remaining:     1
Test error rate:    12.0200002193%
